In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
txn = pd.read_csv('txn.txt', sep='\t')

In [3]:
txn

,customer_id,items,amount_usd,quantity,txn_dt
0,1,A,17,2,2020-11-01
1,1,B,14,1,2020-11-03
2,2,B,14,3,2020-11-09
3,3,C,9,5,2020-11-02
4,3,C,3,8,2020-11-02


In [4]:
statistical_features = txn.groupby('customer_id', as_index=False).agg({
    'amount_usd':['min','median','max','sum','std'],
    'quantity':['min','median','max','sum'],
    'txn_dt' : ['nunique', 'count']
})

In [5]:
statistical_features

customer_id amount_usd                          quantity                 \
                     min median max sum       std      min median max sum   
0           1         14   15.5  17  31  2.121320        1    1.5   2   3   
1           2         14   14.0  14  14       NaN        3    3.0   3   3   
2           3          3    6.0   9  12  4.242641        5    6.5   8  13   

   txn_dt        
  nunique count  
0       2     2  
1       1     1  
2       1     2

# DATETIME

In [6]:
txn['txn_dt'] = pd.to_datetime(txn['txn_dt'])

In [7]:
time_features = txn[['customer_id','txn_dt']].copy()
time_features

,customer_id,txn_dt
0,1,2020-11-01
1,1,2020-11-03
2,2,2020-11-09
3,3,2020-11-02
4,3,2020-11-02


In [8]:
time_features['day'] = time_features['txn_dt'].dt.day
time_features['month'] = time_features['txn_dt'].dt.month
time_features['year'] = time_features['txn_dt'].dt.year
time_features['day_of_week'] = time_features['txn_dt'].dt.dayofweek
time_features['is_month_start'] = time_features['txn_dt'].dt.is_month_start
time_features

,customer_id,txn_dt,day,month,year,day_of_week,is_month_start
0,1,2020-11-01,1,11,2020,6,True
1,1,2020-11-03,3,11,2020,1,False
2,2,2020-11-09,9,11,2020,0,False
3,3,2020-11-02,2,11,2020,0,False
4,3,2020-11-02,2,11,2020,0,False


In [26]:
time_features.groupby('customer_id', as_index=False).agg({
    'day':['nunique'],
    'day_of_week':['nunique'],
    'is_month_start':['mean']
})

,customer_id,day,day_of_week,is_month_start
,,nunique,nunique,mean
0,1,2,2,0.5
1,2,1,1,0.0
2,3,1,1,0.0


# RECENCY

In [9]:
last_txn_dt = time_features.groupby('customer_id', as_index=False)['txn_dt'].max().rename({'txn_dt':'max_txn_dt'},axis=1)
last_txn_dt

,customer_id,max_txn_dt
0,1,2020-11-03
1,2,2020-11-09
2,3,2020-11-02


In [10]:
last_txn_dt['current_dt'] = datetime.datetime.now().date()
last_txn_dt['current_dt'] = pd.to_datetime(last_txn_dt['current_dt'])
last_txn_dt['recency'] = last_txn_dt['current_dt'] - last_txn_dt['max_txn_dt']
last_txn_dt

,customer_id,max_txn_dt,current_dt,recency
0,1,2020-11-03,2020-11-18,15 days
1,2,2020-11-09,2020-11-18,9 days
2,3,2020-11-02,2020-11-18,16 days


# MOB

In [27]:
member_df = pd.DataFrame({
    'customer_id' : [1,2,3],
    'start_member_dt' : ['2010-02-14', '2017-08-19', '2019-01-25']
})
member_df['start_member_dt'] = pd.to_datetime(member_df['start_member_dt'])
member_df

,customer_id,start_member_dt
0,1,2010-02-14
1,2,2017-08-19
2,3,2019-01-25


In [28]:
member_df['current_dt'] = datetime.datetime.now().date()
member_df['current_dt'] = pd.to_datetime(member_df['current_dt'])
member_df['month_on_book'] = (member_df['current_dt'] - member_df['start_member_dt'])/np.timedelta64(1, 'M')

member_df

,customer_id,start_member_dt,current_dt,month_on_book
0,1,2010-02-14,2020-11-18,129.119694
1,2,2017-08-19,2020-11-18,38.998747
2,3,2019-01-25,2020-11-18,21.782788


# Ratio

In [34]:
txn = pd.DataFrame({
    'customer_id' : [1,2,3],
    'income' : [5_000, 10_000, 15_000],
    'spending_amt' : [4_895, 7_500, 10_000]
})
txn

,customer_id,income,spending_amt
0,1,5000,4895
1,2,10000,7500
2,3,15000,10000


In [35]:
txn['utilize_ratio'] = txn['spending_amt'] / txn['income'] 
txn

,customer_id,income,spending_amt,utilize_ratio
0,1,5000,4895,0.979000
1,2,10000,7500,0.750000
2,3,15000,10000,0.666667


# CROSSING

In [36]:
demo = pd.DataFrame({
    'customer_id' : [1,2,3],
    'age_group' : ['21_30', '31_40', '51_60'],
    'wealth_segment' : ['average', 'rich', 'poor']
})
demo

,customer_id,age_group,wealth_segment
0,1,21_30,average
1,2,31_40,rich
2,3,51_60,poor


In [38]:
demo['age_x_wealth'] = demo['age_group'] + '_' + demo['wealth_segment']
demo

,customer_id,age_group,wealth_segment,age_x_wealth
0,1,21_30,average,21_30_average
1,2,31_40,rich,31_40_rich
2,3,51_60,poor,51_60_poor


In [41]:
pd.concat([demo[['customer_id']], pd.get_dummies(demo['age_x_wealth'])],axis=1)

,customer_id,21_30_average,31_40_rich,51_60_poor
0,1,1,0,0
1,2,0,1,0
2,3,0,0,1


# Geo-location

In [65]:
location = pd.DataFrame({
    'customer_id' : [1],
    'latitude' : [13.717674],
    'longitude' : [100.526077]
})
location

,customer_id,latitude,longitude
0,1,13.717674,100.526077


In [66]:
POI = pd.DataFrame({
    'place' : ['hospital','school','hotel'],
    'latitude' : [13.717674, 13.729101, 13.69102],
    'longitude' : [100.519403, 100.512931, 100.4912921]
})
POI

,place,latitude,longitude
0,hospital,13.717674,100.519403
1,school,13.729101,100.512931
2,hotel,13.691020,100.491292


## FINDING THE NEAREST PLACE

- https://en.wikipedia.org/wiki/Haversine_formula
- https://stackoverflow.com/questions/59736682/find-nearest-location-coordinates-in-land-using-python

In [95]:
def dist_between_two_lat_lon(*args):
    from math import asin, cos, radians, sin, sqrt
    # fix argument position
    lat1, long1, lat2, long2 = map(radians, args)

    dist_lats = abs(lat2 - lat1) 
    dist_longs = abs(long2 - long1) 
    a = sin(dist_lats/2)**2 + cos(lat1) * cos(lat2) * sin(dist_longs/2)**2
    c = asin(sqrt(a)) * 2
    radius_earth = 6378 # the "Earth radius" R varies from 6356.752 km at the poles to 6378.137 km at the equator.
    return c * radius_earth

In [101]:
# the code is a bit tedious here
location['distance_to_hospital'] = dist_between_two_lat_lon(
    location.loc[location['customer_id'] == 1]['latitude'],
    location.loc[location['customer_id'] == 1]['longitude'],
    POI.loc[POI['place'] == 'hospital']['latitude'],
    POI.loc[POI['place'] == 'hospital']['longitude'])
location

,customer_id,latitude,longitude,distance_to_hospital
0,1,13.717674,100.526077,0.721739
